# Simple regression models - ShoreShop 2.0

Some simple regression models from old code. These are a nice baseline to explore the data, and get a feel for the process and alongshore behaviour. 
The code I have has the models estimate parameters for dx and then model y as:

`y = jnp.cumsum(f(x)) + e`

Things to deal with:

- check estimate y directly for the simplest models before building on dx style models
- autocorrelation in residuals (though low priority as not fussed about accuracy of uncertainty estimates)


- Check the reason for the years that spike very high and low
  - this appears to be at multiple profiles so I am fairly sure it it real, but need to correlate to certain events
  - seems to be some form of rotation or similar as the profile react together
- Can I use alongshore information in any way - like that was the shoreline predicted at the adjacent profiles last month.
  - spatial lagged autoregression
- NUTS appears the way and not SVI
- try simple onestep ahead vs autoregression

In [ ]:
# magic
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

from functions.data_load import load_modelling_data, tabularise_raw_data

from functions.inference.utils import (
    calc_skill,
    print_skill
)

## Load the data

Load the data using `load_modelling_data()` that returns a dictionary containing the following pandas DataFrames mirroring `1.data_explore.ipynb` (with indices as datetime objects where possible):

- `df_tran`: Transect data.
- `df_gt`: Groundtruth shoreline positions.
- `df_targ_short`: Short-term target shoreline positions.
- `df_targ_medium`: Medium-term target shoreline positions.
- `df_obs`: Observed shoreline positions.
- `dfs_wave`: Wave data for different parameters ('Hs', 'Tp', 'Dir').
- `df_SLR_obs`: Observed sea level rise data.
- `df_SLR_proj`: Projected sea level rise data.

Provided information:
A sequence of shore-normal transects were defined from North to South.\
The distance between transects is 100 m alongshore\
The coordinates of transects were intentionally shifted.

Shoreline positions were retrieved from Landsat 5, 7, 8 and 9 satellite images with [CoastSat toolbox](https://github.com/kvos/CoastSat/tree/master).\
All the shorelines have been corrected to reflect the instaneous position at Mean Sea Level\
Only shorelines from 1987 to 2018 are avaiable for model training/calibration in this site.\
Along each transect, shoreline positions were provided as the distance to the **landward** end of the transect.

Offshore wave data is from [ERA5](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview) wave spectra data.\
SWAN model was used to transfer offshore wave into nearshore with [Binwave](https://www.sciencedirect.com/science/article/pii/S1463500324000337) approach developed by Dr Laura Cagigal.\
Along each transect, the wave data was extracted at 10m contour 

In [ ]:
basedir = '.' # where are you in relation to the root directory
raw_data = load_modelling_data(basedir=basedir)

## Optional preprocessing
Here is provided a template for optional preprocessing steps that can be used to tabularise the data for modelling. Adopted the frequency of the wave data, but this can of course be adjusted.

In [ ]:
# Optional pre-processing
tabular_data = tabularise_raw_data(raw_data)

In [ ]:
sns.scatterplot(tabular_data['df_obs'], hue='Transect', x='date', y='shoreline')

### Quick visualisation
Some quick visualisation of the data to show the format

We have daily data with intermittent shoreline position at 9 transects.

In [ ]:
print(tabular_data.keys())
print(tabular_data['df_obs']['Transect'].unique())
tabular_data['df_obs']

In [ ]:
# pairplot the data
trans_id = 'Transect1'
sns.pairplot(
    tabular_data['df_obs'].query('Transect == @trans_id').drop(columns=['Transect','date'])
)
plt.show()

In [ ]:
# plot the seasonal averages
trans_id = 'Transect1'
plot_data = tabular_data['df_obs'].query('Transect == @trans_id').drop(columns=['Transect']).set_index('date').resample('1ME').mean()
plot_data = plot_data.assign(month=plot_data.index.month)
# demean and stanardise the variables
for _ in [_ for _ in plot_data.columns if not _ in ['month']]:
    plot_data[_] = (plot_data[_] - plot_data[_].mean())/plot_data[_].std()
sns.boxplot(data=plot_data.melt(id_vars='month'),x='month',y='value',hue='variable',boxprops=dict(alpha=.5))
plt.legend(loc=6,bbox_to_anchor=(1.025,0.5))
plt.show()

### Prepare the data
Initially I will resample to monthly to avoid so many NaNs and other unpleasantries. Then we will cheekily gap fill the data, this will be a big exercise in missing data and the choice around how this is handled needs to be considered carefully.

In [ ]:
# calculate how many non NaN values at each transect
print(tabular_data['df_obs'].groupby('Transect').count())

In [ ]:
# split per Transect then average to monthly values with mean and peak for Hs and Tp then recombine into a reasonable dataframe
resampled_data = tabular_data['df_obs'].copy()
resampled_data = pd.concat(
    [
        resampled_data.query('Transect == @trans_id').drop(columns=['Transect']).set_index('date').resample('MS').agg({'Hs':['mean','max'],'Tp':['mean','max'],'Dir':['mean'],'shoreline':['mean']}).reset_index().assign(Transect=trans_id) for trans_id in resampled_data['Transect'].unique()
    ], axis=0
)
# combine the column names to make one level
resampled_data.columns = [
    '_'.join(col).strip() if '' != col[1] else col[0] for col in resampled_data.columns.values]
resampled_data = resampled_data.rename(columns={'shoreline_mean':'shoreline'})
# now add month predictor
resampled_data = resampled_data.assign(month=resampled_data['date'].dt.month-1)
resampled_data = resampled_data.reset_index(drop=True)
# get the shoreline position at t-1 but only for the same transect
resampled_data = resampled_data.assign(shoreline_tminus1=resampled_data.groupby('Transect')['shoreline'].shift(1))
# Keep only Transect6
resampled_data = resampled_data.query('Transect == "Transect7"').reset_index(drop=True)
resampled_data

In [ ]:
# vis of transect data
fig = plt.figure(figsize=(16,8))
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)
sns.lineplot(data=tabular_data['df_obs'],y='shoreline',x='date', hue='Transect', ax=ax1, alpha=0.25)
sns.lineplot(data=tabular_data['df_obs'],y='shoreline',x='date', ax=ax1, color = 'k', errorbar=None)

sns.lineplot(data=resampled_data,y='shoreline', x='date', hue='Transect', ax=ax2, alpha=0.25)
sns.lineplot(data=resampled_data,y='shoreline', x='date', ax=ax2, color = 'k', errorbar=None)
sns.lineplot(data=resampled_data.query('Transect == "Transect1"'),y='shoreline', x='date', color = 'C0', ax=ax2, alpha=0.75)
sns.lineplot(data=resampled_data.query('Transect == "Transect9"'),y='shoreline', x='date', color = 'C3', ax=ax2, alpha=0.75)
plt.show()



In [ ]:
# Split into training 1999-2016 and testing 2017-2019
train_data = resampled_data.query('date < "2017-01-01"').reset_index(drop=True)
test_data = resampled_data.query('date >= "2017-01-01"').reset_index(drop=True)

# Create the modelling copies

In [ ]:
train_data_clean = train_data.dropna().copy()
test_data_clean = test_data.dropna().copy()
train_data_out = train_data_clean.copy()
test_data_out = test_data_clean.copy()

display(train_data_clean)
display(test_data_clean)

## Modelling attempt
Here you can construct your model and make some predictions.

In [ ]:
# Assuming train_data and test_data are already defined DataFrames
# Define the feature matrix X and target vector y
X_train = train_data_clean.drop(columns=['date', 'shoreline', 'Transect'])
y_train = train_data_clean['shoreline']
X_test = test_data_clean.drop(columns=['date', 'shoreline', 'Transect'])

# Initialize the scalers
scaler_X = StandardScaler()
scaler_y = StandardScaler()

# Scale the feature matrix and target vector
X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))

# Create and train the regression model
model = RandomForestRegressor()
model.fit(X_train_scaled, y_train_scaled)

# Prepare the test data
X_test_scaled = scaler_X.transform(X_test)

# Make predictions on the test data
predictions_scaled = model.predict(X_test_scaled)
# Rescale the predictions back to the original scale
predictions = scaler_y.inverse_transform(predictions_scaled.reshape(-1, 1))

# If you want to add the predictions to the test_data DataFrame
test_data_out['predicted_shoreline'] = predictions

# Make predictions on the training data
predictions_scaled_train = model.predict(X_train_scaled)
# Rescale the predictions back to the original scale
predictions_train = scaler_y.inverse_transform(predictions_scaled_train.reshape(-1, 1))
train_data_out['predicted_shoreline'] = predictions_train

# Display the first few rows of the test_data with predictions
print(train_data_out.head())
print(test_data_out.head())

# Plot predicted

In [ ]:
# Scatter plot of the predicted vs. observed shoreline positions
fig = plt.figure(figsize=(8, 4))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
sns.scatterplot(data=test_data_out, x='shoreline', y='predicted_shoreline', ax=ax1)
sns.scatterplot(data=train_data_out, x='shoreline', y='predicted_shoreline', ax=ax2)

ax1.set_title('Test data')
ax2.set_title('Train data')

ax1.set_xlabel('Observed shoreline position')
ax1.set_ylabel('Predicted shoreline position')
ax2.set_xlabel('Observed shoreline position')
ax2.set_ylabel('Predicted shoreline position')

plt.subplots_adjust(wspace=0.25)
plt.show()

In [ ]:
# Plot timeseries of observed and predicted shoreline positions
plt.figure(figsize=(8, 3))
sns.lineplot(data=train_data, x='date', y='shoreline', label='Observed shoreline position')
sns.lineplot(data=train_data_out, x='date', y='predicted_shoreline', label='Predicted shoreline position')
plt.ylabel('Shoreline position')
plt.title('Observed and predicted shoreline positions - Training data')
plt.show()

In [ ]:
# Plot timeseries of observed and predicted shoreline positions
plt.figure(figsize=(8, 3))
sns.lineplot(data=test_data, x='date', y='shoreline', label='Observed shoreline position')
sns.lineplot(data=test_data_out, x='date', y='predicted_shoreline', label='Predicted shoreline position')
plt.ylabel('Shoreline position')
plt.title('Observed and predicted shoreline positions - Test data')
plt.show()

### Compare model skill
Compare the skill of each model by RMSE, R2 (of dShl), BSS.

In [ ]:
# report the RMSE, BSS and R2
print('#'*80)
print('Train - one step')
rmse, r2, bss, r = calc_skill(
    curr_obs=train_data_out['shoreline'],
    prev_obs=train_data_out['shoreline_tminus1'],
    mean_mu=train_data_out['predicted_shoreline'].values
)
print('BSS: {:.2f} | RMSE: {:.2f} | R2: {:.2f} | r: {:.2f}'.format(bss,rmse,r2,r))
print('#'*80)
print('Test - one step')
rmse, r2, bss, r = calc_skill(
    curr_obs=test_data_out['shoreline'],
    prev_obs=test_data_out['shoreline_tminus1'],
    mean_mu=test_data_out['predicted_shoreline'].values
)
print('BSS: {:.2f} | RMSE: {:.2f} | R2: {:.2f} | r: {:.2f}'.format(bss,rmse,r2,r))